In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip "/content/drive/MyDrive/facedata_croped.zip" -d"/content/drive/MyDrive/FaceNet_own_data"

In [3]:
# !pip install facenet-pytorch

In [1]:
# !pip install torch torchvision torchaudio
# !pip install mtcnn pillow numpy scikit-learn


In [2]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC

In [ ]:
base_path = r"C:\Users\nasir\OneDrive\Desktop\facedata_croped"
all_identities = os.listdir(base_path)[:]  
print(all_identities)

['Anthony Mackie', 'Antoine Griezmann', 'Benedict Cumberbatch', 'Brie Larson', 'Bruno Fernandes', 'Chris Evans', 'Chris Hemsworth', 'Cristiano Ronaldo', 'Danai Gurira', 'Dave Bautista', 'Don Cheadle', 'Elizabeth Olsen', 'Erling Haaland', 'Florence Pugh', 'Gwyneth Paltrow', 'Harry Kane', 'Hayley Atwell', 'Jeremy Renner', 'Joshua Kimmich', 'João Félix', 'Jude Bellingham', 'Karen Gillan', 'Karim Benzema', 'Kevin De Bruyne', 'Kylian Mbappé', 'Letitia Wright', 'Lionel Messi', 'Luka Modrić', 'Marcus Rashford', 'Mark Ruffalo', 'Mohamed Salah', 'Neymar', 'Paul Rudd', 'Paulo Dybala', 'Pedri', 'Phil Foden', 'Robert Downey Jr', 'Robert Lewandowski', 'Romelu Lukaku', 'Sadio Mané', 'Samuel L. Jackson', 'Scarlett Johansson', 'Sebastian Stan', 'Son Heung-min', 'Tom Hiddleston', 'Tom Holland', 'Vinícius Júnior', 'Virgil van Dijk', 'Zoe Saldaña']


In [5]:

mtcnn = MTCNN(image_size=160, margin=0)
def extract_face(img_path):
    img = Image.open(img_path).convert('RGB')
    face = mtcnn(img)
    if face is not None:
        return face
    return None


In [6]:
data = []
labels = []
for label, identity in enumerate(all_identities):
    identity_path = os.path.join(base_path, identity)
    images = os.listdir(identity_path)[:]
    for img_name in images:
        img_path = os.path.join(identity_path, img_name)
        face = extract_face(img_path)
        if face is not None:
            data.append(face)
            labels.append(label)
print(f"Total faces loaded: {len(data)}")


Total faces loaded: 1148


In [7]:
faces = torch.stack(data)
labels = np.array(labels)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:

facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

embeddings = []
with torch.no_grad():
    for face in faces:
        face = face.unsqueeze(0).to(device)
        embedding = facenet(face)
        embeddings.append(embedding.cpu().numpy())
embeddings = np.vstack(embeddings)


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [9]:
# Normalize embeddings
in_encoder = Normalizer(norm='l2')
embeddings_norm = in_encoder.transform(embeddings)

# Encode labels
out_encoder = LabelEncoder()
labels_enc = out_encoder.fit_transform(labels)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    embeddings_norm, labels_enc, test_size=0.2, random_state=42, stratify=labels_enc
)

model_svm = SVC(kernel='linear', probability=True)
model_svm.fit(X_train, y_train)
print("SVM classifier trained.")




SVM classifier trained.


In [11]:
acc = model_svm.score(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


Test Accuracy: 0.9130


In [ ]:
## prediction
def predict_identity(image_path, facenet, model_svm, in_encoder, out_encoder, mtcnn, device):
    img = Image.open(image_path).convert('RGB')
    face = mtcnn(img)
    if face is not None:
        face = face.unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = facenet(face).cpu().numpy()
        embedding = in_encoder.transform(embedding)
        yhat_class = model_svm.predict(embedding)
        yhat_prob = model_svm.predict_proba(embedding)
        class_index = yhat_class[0]
        class_probability = yhat_prob[0, class_index] * 100
        predict_name = out_encoder.inverse_transform(yhat_class)
        return predict_name[0], class_probability
    else:
        return None, None

# Example usage
image_path = '/content/Screenshot 2025-07-04 185305.png'
identity, confidence = predict_identity(
    image_path, facenet, model_svm, in_encoder, out_encoder, mtcnn, device
)
if identity is not None:
    print(f'Predicted Identity: {all_identities[identity]}, Confidence: {confidence:.2f}')
else:
    print("No face detected or error occurred.")


In [12]:
import joblib

# Save the trained components
joblib.dump(model_svm, 'svm_model.pkl')
joblib.dump(in_encoder, 'in_encoder.pkl')
joblib.dump(out_encoder, 'out_encoder.pkl')
joblib.dump(all_identities, 'all_identities.pkl')

['all_identities.pkl']